In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pycountry

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file.")

client = OpenAI(api_key = openai_api_key)

with open("visa_information.json", "r") as file:
    visa_data = json.load(file)

In [ ]:
def get_country_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

In [ ]:
def check_visa(passport_country_name, destination_country_name):
    passport_code = get_country_code(passport_country_name)
    dest_code = get_country_code(destination_country_name)

    if not passport_code or not dest_code:
        return {"status": "unknownCountry", "max_stay": None}

    if dest_code not in visa_data:
        return {"status": "applyOffline", "max_stay": None}

    destination_info = visa_data[dest_code]

    if "visaFree" in destination_info and passport_code in destination_info["visaFree"]:
        max_stay = destination_info["visaFree"][passport_code].get("maxStay")
        return {"status": "visaFree", "max_stay": max_stay}

    if "visaOnArrival" in destination_info and passport_code in destination_info["visaOnArrival"]:
        max_stay = destination_info["visaOnArrival"][passport_code].get("maxStay")
        return {"status": "visaOnArrival", "max_stay": max_stay}

    return {"status": "applyOffline", "max_stay": None}

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_visa_requirements",
            "description": "Determines visa requirements (visa type, max stay) for travel between two countries based ONLY on provided data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "passport_country_name": {
                        "type": "string",
                        "description": "The traveler's passport issuing country e.g. Lebanon, Singapore.",
                    },
                    "destination_country_name": {
                        "type": "string",
                        "description": "The country the traveler wants to visit e.g. Portugal.",
                    },
                },
                "required": ["passport_country_name", "destination_country_name"],
            },
        }
    }
]

model_name = "gpt-4o-mini" 

system_prompt = """You are a visa assistant. Follow these steps:
- Ask for passport country and destination if missing
- Use only the provided function for visa checks
- Thank the user for providing the info and respond based on these statuses:
   - visaFree: Let the user know they don’t need a visa and mention the maximum stay in days if available.
   - visaOnArrival: Let the user know they can get a visa on arrival and mention the maximum stay in days if available.
   - applyOffline: Let the user know they need to apply for a visa before traveling, stating that visa-free or visa-on-arrival options are not available.
   - unknownCountry: Let the user know you couldn't recognize one of the countries and ask them to clarify.
- Do not provide any visa information based on your own knowledge, only use the result from the function call.
- Keep responses concise, friendly, and professional"""

In [ ]:
def run_conversation():
    messages = [{"role": "system", "content": system_prompt}]
    print("\nAI: Hello! How can I help you with visa requirements today? Please tell me your passport nationality and destination country.")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("AI: Goodbye!")
            break

        messages.append({"role": "user", "content": user_input})

        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages,
                tools=tools,
                tool_choice="auto",
            )
            response_message = response.choices[0].message

            tool_calls = response_message.tool_calls

            if tool_calls:
                messages.append(response_message)
                
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    
                    if function_name == "get_visa_requirements":
                        try:
                            function_args = json.loads(tool_call.function.arguments)
                            
                            function_response = check_visa(
                                passport_country_name=function_args.get("passport_country_name"),
                                destination_country_name=function_args.get("destination_country_name")
                            )

                            messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": json.dumps(function_response),
                            })
                        except Exception as e:
                            error_message = f"Error processing request: {str(e)}"
                            messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": json.dumps({"error": error_message}),
                            })

                second_response = client.chat.completions.create(
                    model=model_name,
                    messages=messages,
                )
                final_response_message = second_response.choices[0].message
                print(f"AI: {final_response_message.content}")
                messages.append(final_response_message)

            else:
                assistant_response = response_message.content
                print(f"AI: {assistant_response}")
                messages.append({"role": "assistant", "content": assistant_response})

        except Exception as e:
            print("AI: Sorry, something went wrong. Please try rephrasing your request.")

In [ ]:
run_conversation()

You:  hello


AI: Hello! How can I assist you today? If you need information regarding visas, please provide your passport country and destination country.


You:  I want to travel to greece from portugal


AI: Could you please provide your passport country? This will help me check the visa requirements for your travel to Greece. Thank you!


You:  told you 
